# Import Required Libraries
Import necessary libraries such as argparse, math, collections, numpy, Metashape, os, sys, exifread, importlib, csv, and Path from pathlib.

In [1]:
import argparse
import math
import collections
import numpy as np
import Metashape
import os
import sys
import exifread
from collections import defaultdict
from upd_micasense_pos import ret_micasense_pos
import importlib
import upd_micasense_pos
import csv
from pathlib import Path

importlib.reload(upd_micasense_pos)

<module 'upd_micasense_pos' from 'c:\\Users\\admin\\Documents\\Python Scripts\\drone_metashape\\upd_micasense_pos.py'>

# Define Constants and Global Variables
Define constants and global variables used throughout the script, such as BASE_DIR, use_model, use_dem, GEOG_COORD, SOURCE_CRS, CONST_a, CONST_inv_f, CHUNK_RGB, CHUNK_MULTISPEC, IMG_QUAL_THRESHOLD, DICT_SMOOTH_STRENGTH, P1_GIMBAL1_OFFSET, and offset_dict.

In [2]:
# Define Constants and Global Variables

# Base directory for the project
BASE_DIR = "M:/working_package_2/2024_dronecampaign/01_data/dronetest/processing_test"

# Flags to decide whether to use model or DEM for orthomosaic
use_model = False
use_dem = True

# Named tuple for geographic coordinates
GEOG_COORD = collections.namedtuple('Geog_CS', ['lat_decdeg', 'lon_decdeg', 'elliph'])

# Source coordinate reference system (CRS)
SOURCE_CRS = Metashape.CoordinateSystem("EPSG::4326")  # WGS84

# Constants for WGS84 ellipsoid
CONST_a = 6378137  # Semi major axis
CONST_inv_f = 298.257223563  # Inverse flattening 1/f WGS84 ellipsoid

# Chunk labels in Metashape
CHUNK_RGB = "rgb"
CHUNK_MULTISPEC = "multispec"

# Image quality threshold
IMG_QUAL_THRESHOLD = 0.7

# Dictionary for smoothing strength values
DICT_SMOOTH_STRENGTH = {'low': 50, 'medium': 100, 'high': 200}

# Lever-arm offsets for different sensors on Matrice 300
P1_GIMBAL1_OFFSET = (0.087, 0.0, 0.0)

# Lever-arm offsets for MicaSense sensors
offset_dict = defaultdict(dict)
offset_dict['RedEdge-M']['Red'] = (-0.097, -0.03, -0.06)
offset_dict['RedEdge-M']['Dual'] = (-0.097, 0.02, -0.08)
offset_dict['RedEdge-P']['Red'] = (0, 0, 0)
offset_dict['RedEdge-P']['Dual'] = (0, 0, 0)

# Define Helper Functions
Define helper functions used in the script, including `cartesian_to_geog`, `find_files`, and `copyBoundingBox`.

In [ ]:
# Define Helper Functions

def cartesian_to_geog(X, Y, Z):
    """
    Convert Cartesian coordinates to geographic coordinates using WGS84 ellipsoid.
    Return Lat, Lon, ellipsoidal height as a named tuple.
    """
    f = 1 / CONST_inv_f
    e_sq = 2 * f - f ** 2
    p = math.sqrt(X ** 2 + Y ** 2)
    r = math.sqrt(p ** 2 + Z ** 2)
    mu = math.atan((Z / p) * (1 - f) + (e_sq * CONST_a) / r)

    lat_top_line = Z * (1 - f) + e_sq * CONST_a * math.sin(mu) ** 3
    lat_bottom_line = (1 - f) * (p - e_sq * CONST_a * math.cos(mu) ** 3)

    lon = math.atan(Y / X)
    lat = math.atan(lat_top_line / lat_bottom_line)

    if lon < 0:
        tmp_lon = lon + math.pi
    else:
        tmp_lon = lon

    lon_dec_deg = (tmp_lon / math.pi) * 180
    lat_dec_deg = (lat / math.pi) * 180

    ellip_h = p * math.cos(lat) + Z * math.sin(lat) - CONST_a * math.sqrt(1 - e_sq * math.sin(lat) ** 2)

    conv_coord = GEOG_COORD(lat_dec_deg, lon_dec_deg, ellip_h)

    return conv_coord

def find_files(folder, types):
    """
    Find files of specified types in a folder.
    """
    photo_list = list()
    for dir, subdir, file in os.walk(folder):
        for filename in file:
            if filename.lower().endswith(types):
                photo_list.append(os.path.join(dir, filename))
    return photo_list

def copyBoundingBox(from_chunk_label, to_chunk_labels):
    """
    Copy bounding box from one chunk to others.
    """
    print("Script started...")

    doc = Metashape.app.document()

    from_chunk = None
    for chunk in doc.chunks:
        if chunk.label == from_chunk_label:
            from_chunk = chunk
            break

    if not from_chunk:
        print(f"Chunk with label '{from_chunk_label}' not found.")
        return

    to_chunks = []
    for chunk in doc.chunks:
        if chunk.label in to_chunk_labels:
            to_chunks.append(chunk)

    if not to_chunks:
        print("No valid target chunks found.")
        return

    print("Copying bounding box from chunk '" + from_chunk.label + "' to " + str(len(to_chunks)) + " chunks...")

    T0 = from_chunk.transform.matrix

    region = from_chunk.region
    R0 = region.rot
    C0 = region.center
    s0 = region.size

    for chunk in to_chunks:
        if chunk == from_chunk:
            continue

        T = chunk.transform.matrix.inv() * T0

        R = Metashape.Matrix([[T[0, 0], T[0, 1], T[0, 2]],
                              [T[1, 0], T[1, 1], T[1, 2]],
                              [T[2, 0], T[2, 1], T[2, 2]]])

        scale = R.row(0).norm()
        R = R * (1 / scale)

        new_region = Metashape.Region()
        new_region.rot = R * R0
        c = T.mulp(C0)
        new_region.center = c
        new_region.size = s0 * scale / 1.

        chunk.region = new_region

# Define Main Processing Functions
Define the main processing functions `proc_rgb` and `proc_multispec` that handle the processing of RGB and multispectral images, respectively.

In [6]:
# Define Main Processing Functions

def proc_rgb():
    """
    Process RGB images to create orthomosaic and 3D model.
    """
    chunk = doc.findChunk(dict_chunks[CHUNK_RGB])
    proj_file = doc.path
    blockshift_p1 = False

    if args.drtk is not None:
        blockshift_p1 = True
        DRTK_TXT_FILE = args.drtk
        print("P1 blockshift set")

        with open(DRTK_TXT_FILE, 'r') as file:
            line = file.readline()
            split_line = line.split(',')
            drtk_field = cartesian_to_geog(float(split_line[0]), float(split_line[1]), float(split_line[2]))
            line = file.readline()
            split_line = line.split(',')
            drtk_auspos = cartesian_to_geog(float(split_line[0]), float(split_line[1]), float(split_line[2]))

        diff_lat = round((drtk_auspos.lat_decdeg - drtk_field.lat_decdeg), 6)
        diff_lon = round((drtk_auspos.lon_decdeg - drtk_field.lon_decdeg), 6)
        diff_elliph = round((drtk_auspos.elliph - drtk_field.elliph), 6)
        P1_shift = Metashape.Vector((diff_lon, diff_lat, diff_elliph))

        print("Shifting P1 cameras by: " + str(P1_shift))

        for camera in chunk.cameras:
            if not camera.label == camera.master.label:
                continue
            if not camera.reference.location:
                continue
            else:
                camera.reference.location = camera.reference.location + P1_shift

    target_crs = Metashape.CoordinateSystem("EPSG::" + args.crs)
    for camera in chunk.cameras:
        if not camera.reference.location:
            continue
        camera.reference.location = Metashape.CoordinateSystem.transform(camera.reference.location, SOURCE_CRS, target_crs)

    chunk.crs = target_crs

    global P1_shift_vec
    if blockshift_p1:
        chunk.exportReference(path=str(P1_CAM_CSV), format=Metashape.ReferenceFormatCSV, columns="nxyz", delimiter=",", items=Metashape.ReferenceItemsCameras)
        P1_shift_vec = np.array([diff_lat, diff_lon, diff_elliph])
    else:
        P1_shift_vec = np.array([0.0, 0.0, 0.0])

    doc.save()

    if METASHAPE_V2_PLUS:
        chunk.analyzeImages()
    else:
        chunk.analyzePhotos()
    low_img_qual = [camera for camera in chunk.cameras if (float(camera.meta["Image/Quality"]) < IMG_QUAL_THRESHOLD)]
    if low_img_qual:
        print("Removing cameras with Image Quality < %.1f" % IMG_QUAL_THRESHOLD)
        chunk.remove(low_img_qual)
    doc.save()

    print(chunk.sensors[0].antenna.location_ref)
    print("Update GPS/INS offset for P1")
    chunk.sensors[0].antenna.location_ref = Metashape.Vector(P1_GIMBAL1_OFFSET)
    print(chunk.sensors[0].antenna.location_ref)

    print("Aligning Cameras")
    chunk.camera_location_accuracy = Metashape.Vector((0.10, 0.10, 0.10))
    chunk.matchPhotos(downscale=quality1, generic_preselection=False, reference_preselection=True, reference_preselection_mode=Metashape.ReferencePreselectionSource)
    chunk.alignCameras()
    doc.save()

    print("Gradual selection for reprojection error...")
    f = Metashape.TiePoints.Filter()
    threshold = 0.5
    f.init(chunk, criterion=Metashape.TiePoints.Filter.ReprojectionError)
    f.removePoints(threshold)
    doc.save()

    print("Optimise alignment")
    chunk.optimizeCameras()
    doc.save()

    print("Build dense cloud")
    chunk.buildDepthMaps(downscale=quality2)
    doc.save()

    if METASHAPE_V2_PLUS:
        chunk.buildPointCloud()
    else:
        chunk.buildDenseCloud()
    doc.save()

    if use_model:
        print("Build mesh")
        if METASHAPE_V2_PLUS:
            chunk.buildModel(surface_type=Metashape.HeightField, source_data=Metashape.PointCloudData, face_count=Metashape.MediumFaceCount)
        else:
            chunk.buildModel(surface_type=Metashape.HeightField, source_data=Metashape.DenseCloudData, face_count=Metashape.MediumFaceCount)
        doc.save()

        chunk.decimateModel(face_count=len(chunk.model.faces) / 2)
        smooth_val = DICT_SMOOTH_STRENGTH[args.smooth]
        chunk.smoothModel(smooth_val)
        model_file = Path(proj_file).parent / (Path(proj_file).stem + "_rgb_smooth_" + str(smooth_val) + ".obj")
        chunk.exportModel(path=str(model_file), crs=target_crs, format=Metashape.ModelFormatOBJ)

    compression = Metashape.ImageCompression()
    compression.tiff_compression = Metashape.ImageCompression.TiffCompressionLZW
    compression.tiff_big = True
    compression.tiff_tiled = True
    compression.tiff_overviews = True

    if use_dem:
        print("Build DEM")
        dem_res_xy = 0.01
        if METASHAPE_V2_PLUS:
            chunk.buildDem(source_data=Metashape.PointCloudData, resolution=dem_res_xy)
        else:
            chunk.buildDem(source_data=Metashape.DenseCloudData, resolution=dem_res_xy)
        doc.save()

        dem_file = Path(proj_file).parent / (Path(proj_file).stem + "_dem_" + str(dem_res_xy).split('.')[1] + ".tif")
        chunk.exportRaster(path=dem_file, source_data=Metashape.ElevationData, image_format=Metashape.ImageFormatTIFF, image_compression=compression)

    test = args.test

    if not test:
        print("Build orthomosaic")
        chunk.buildOrthomosaic(surface_data=Metashape.DataSource.ModelData, refine_seamlines=True)
        doc.save()

        if chunk.orthomosaic:
            res_xy = 0.01
            p1_idx = MRK_PATH.find("rgb")
            if p1_idx == -1:
                dir_path = Path(proj_file).parent
                print("Cannot find rgb/ folder. Saving ortho in " + str(dir_path))
            else:
                dir_path = Path(MRK_PATH[:p1_idx + len("rgb")]) / "level1_proc"
                dir_path.mkdir(parents=True, exist_ok=True)

            ortho_file = dir_path / (Path(proj_file).stem + "_rgb_ortho_" + str(res_xy).split('.')[1] + ".tif")
            chunk.exportRaster(path=str(ortho_file), resolution_x=res_xy, resolution_y=res_xy, image_format=Metashape.ImageFormatTIFF, save_alpha=False, source_data=Metashape.OrthomosaicData, image_compression=compression)
            print("Exported orthomosaic " + str(ortho_file))
        else:
            print("Skipping orthomosaic building and exporting due to test mode.")

        report_path = dir_path / (Path(proj_file).stem + "_rgb_report.pdf")
        print(f"Exporting processing report to {report_path}...")
        chunk.exportReport(path=str(report_path))
        doc.save()

        print("RGB chunk processing complete!")

def proc_multispec():
    """
    Process multispectral images to create orthomosaic with relative reflectance.
    """
    chunk = doc.findChunk(dict_chunks[CHUNK_MULTISPEC])
    target_crs = Metashape.CoordinateSystem("EPSG::" + args.crs)
    camera = chunk.cameras[0]
    cam_master = camera.master.label.split('_')
    img_suffix_master = cam_master[2]

    global P1_shift_vec
    if args.multionly:
        P1_shift_vec = np.array([0.0, 0.0, 0.0])

    print("Interpolate Micasense position based on P1 with blockshift" + str(P1_shift_vec))
    ret_micasense_pos(MRK_PATH, MICASENSE_PATH, img_suffix_master, args.crs, str(MICASENSE_CAM_CSV), P1_shift_vec)
    chunk.importReference(str(MICASENSE_CAM_CSV), format=Metashape.ReferenceFormatCSV, columns="nxyz", delimiter=",", crs=target_crs, skip_rows=1, items=Metashape.ReferenceItemsCameras)
    doc.save()

    del_camera_names = list()
    for camera in chunk.cameras:
        if not camera.label == camera.master.label:
            continue
        if not camera.reference.location:
            continue
        if camera.reference.location.z == 0:
            del_camera_names.append(camera.label)

    print("Deleting MicaSense images that triggered outside P1 capture times")
    for camera in chunk.cameras:
        if camera.group is not None:
            if camera.group.label == 'Calibration images':
                continue
        if camera.label in del_camera_names:
            chunk.remove(camera)
    doc.save()

    if cam_model == 'RedEdge-M':
        set_primary = "NIR"
    elif cam_model == 'RedEdge-P':
        set_primary = 'Panchro'
    for s in chunk.sensors:
        if s.label.find(set_primary) != -1:
            print("Setting primary channel to " + s.label)
            chunk.primary_channel = s.layer_index
            break

    print("Updating Micasense GPS offset")
    chunk.sensors[0].antenna.location_ref = Metashape.Vector(MS_GIMBAL2_OFFSET)

    print("Updating Raster Transform for relative reflectance")
    raster_transform_formula = []
    num_bands = len(chunk.sensors)
    if cam_model == 'RedEdge-M':
        for band in range(1, num_bands + 1):
            raster_transform_formula.append("B" + str(band) + "/32768")
    elif cam_model == 'RedEdge-P':
        if num_bands >= 10:
            PANCHRO_BAND = 5
        else:
            PANCHRO_BAND = 3
        for band in range(1, num_bands + 1):
            if band != PANCHRO_BAND:
                raster_transform_formula.append("B" + str(band) + "/32768")

    chunk.raster_transform.formula = raster_transform_formula
    chunk.raster_transform.calibrateRange()
    chunk.raster_transform.enabled = True
    doc.save()

    if METASHAPE_V2_PLUS:
        chunk.analyzeImages()
    else:
        chunk.analyzePhotos()
    low_img_qual = [camera.master for camera in chunk.cameras if (float(camera.meta["Image/Quality"]) < 0.5)]
    if low_img_qual:
        print("Removing cameras with Image Quality < %.1f" % 0.5)
        chunk.remove(list(set(low_img_qual)))
    doc.save()

    chunk.calibrateReflectance(use_reflectance_panels=True, use_sun_sensor=args.sunsens)

    chunk.camera_location_accuracy = Metashape.Vector((0.10, 0.10, 0.10))
    chunk.matchPhotos(downscale=quality3, generic_preselection=False, reference_preselection=True, reference_preselection_mode=Metashape.ReferencePreselectionSource)
    doc.save()
    print("Aligning cameras")
    chunk.alignCameras()
    doc.save()

    print("Gradual selection for reprojection error...")
    f = Metashape.TiePoints.Filter()
    threshold = 0.5
    f.init(chunk, criterion=Metashape.TiePoints.Filter.ReprojectionError)
    f.removePoints(threshold)
    doc.save()

    print("Optimise alignment")
    chunk.optimizeCameras()
    doc.save()

    copyBoundingBox(CHUNK_RGB, CHUNK_MULTISPEC)

    if use_model:
        smooth_val = DICT_SMOOTH_STRENGTH[args.smooth]
        model_file = Path(proj_file).parent / (Path(proj_file).stem + "_rgb_smooth_" + str(smooth_val) + ".obj")
        chunk.importModel(path=str(model_file), crs=target_crs, format=Metashape.ModelFormatOBJ)

        print("Build orthomosaic")
        chunk.buildOrthomosaic(surface_data=Metashape.DataSource.ModelData, refine_seamlines=True)
        doc.save()

    if use_dem:
        dem_res_xy = 0.01
        dem_file = Path(proj_file).parent / (Path(proj_file).stem + "_dem_" + str(dem_res_xy).split('.')[1] + ".tif")
        chunk.importRaster(path=dem_file, crs=target_crs, format=Metashape.ImageFormatTIFF)

        print("Build orthomosaic")
        chunk.buildOrthomosaic(surface_data=Metashape.DataSource.ElevationData, refine_seamlines=True)
        doc.save()

    if chunk.orthomosaic:
        res_xy = 0.05
        micasense_idx = MICASENSE_PATH.find("multispec")
        if micasense_idx == -1:
            dir_path = Path(proj_file).parent
            print("Cannot find " + "multispec/ folder. Saving ortho in " + str(dir_path))
        else:
            dir_path = Path(MICASENSE_PATH[:micasense_idx + len("multispec")]) / "level1_proc"
            dir_path.mkdir(parents=True, exist_ok=True)

        ortho_file = dir_path / (Path(proj_file).stem + "_" + "multispec_ortho_" + str(res_xy).split('.')[1] + ".tif")
        compression = Metashape.ImageCompression()
        compression.tiff_compression = Metashape.ImageCompression.TiffCompressionLZW
        compression.tiff_big = True
        compression.tiff_tiled = True
        compression.tiff_overviews = True

        chunk.exportRaster(path=str(ortho_file), resolution_x=res_xy, resolution_y=res_xy, image_format=Metashape.ImageFormatTIFF, raster_transform=Metashape.RasterTransformValue, save_alpha=False, source_data=Metashape.OrthomosaicData, image_compression=compression)
        print("Exported orthomosaic: " + str(ortho_file))

    report_path = dir_path / (Path(proj_file).stem + "_multispec_report.pdf")
    print(f"Exporting processing report to {report_path}...")
    chunk.exportReport(path=str(report_path))
    doc.save()

    print("Multispec chunk processing complete!")

# Define Argument Parsing and Initialization
Set up argument parsing using argparse and initialize variables such as MRK_PATH, MICASENSE_PATH, and doc.

In [ ]:
import argparse

# Simulate command-line arguments
args_list = [
    '-date', '20240723',
    '-site', 'Site_test5',
    '-crs', '2056',
    '-multispec', 'M:\\working_package_2\\2024_dronecampaign\\01_data\\dronetest\\MicasenseData\\fullset',
    '-rgb', 'M:\\working_package_2\\2024_dronecampaign\\01_data\\dronetest\\P1Data\\DJI_202408080937_002_p1micasense60mtest',
    '-smooth', 'low',
    '-test', 'True'
]

# Parse arguments
parser = argparse.ArgumentParser(description='Update camera positions in P1 and/or MicaSense chunks in Metashape project')
parser.add_argument('-proj_path', help='path to Metashape project file')
parser.add_argument('-date', help='Date of flight in YYYYMMDD format', required=True)
parser.add_argument('-site', help='Site name', required=True)
parser.add_argument('-crs', help='EPSG code for target projected CRS for micasense cameras. E.g: 7855 for GDA2020/MGA zone 55', required=True)
parser.add_argument('-multispec', help='path to multispectral level0_raw folder with raw images')
parser.add_argument('-rgb', help='path to RGB level0_raw folder that also has the MRK files')
parser.add_argument('-smooth', help='Smoothing strength used to smooth RGB mesh low/med/high', default="low")
parser.add_argument('-drtk', help='If RGB coordinates to be blockshifted, file containing DRTK base station coordinates from field and AUSPOS', default=None)
parser.add_argument('-sunsens', help='boolean to use sun sensor data for reflectance calibration', default=False)
parser.add_argument('-test', help='boolean to make processing faster for debugging', default=False)
parser.add_argument('-multionly', help='boolean to process multispec chunk only', default=False)

args = parser.parse_args(args_list)

# Print parsed arguments to verify
print(args)

Namespace(proj_path=None, date='20240723', site='Site_test5', crs='2056', multispec='M:\\working_package_2\\2024_dronecampaign\\01_data\\dronetest\\MicasenseData\\fullset', rgb='M:\\working_package_2\\2024_dronecampaign\\01_data\\dronetest\\P1Data\\DJI_202408080937_002_p1micasense60mtest', smooth='low', drtk=None, sunsens=False, test=False, multionly=False)


In [ ]:
# Define Argument Parsing and Initialization

# Parse arguments and initialize variables
parser = argparse.ArgumentParser(description='Update camera positions in P1 and/or MicaSense chunks in Metashape project')
parser.add_argument('-proj_path', help='path to Metashape project file')
parser.add_argument('-date', help='Date of flight in YYYYMMDD format', required=True)
parser.add_argument('-site', help='Site name', required=True)
parser.add_argument('-crs', help='EPSG code for target projected CRS for micasense cameras. E.g: 7855 for GDA2020/MGA zone 55', required=True)
parser.add_argument('-multispec', help='path to multispectral level0_raw folder with raw images')
parser.add_argument('-rgb', help='path to RGB level0_raw folder that also has the MRK files')
parser.add_argument('-smooth', help='Smoothing strength used to smooth RGB mesh low/med/high', default="low")
parser.add_argument('-drtk', help='If RGB coordinates to be blockshifted, file containing DRTK base station coordinates from field and AUSPOS', default=None)
parser.add_argument('-sunsens', help='boolean to use sun sensor data for reflectance calibration', default=False)
parser.add_argument('-test', help='boolean to make processing faster for debugging', default=False)
parser.add_argument('-multionly', help='boolean to process multispec chunk only', default=False)

args = parser.parse_args(args_list)

# Initialize global variables
MRK_PATH = args.rgb if args.rgb else None
MICASENSE_PATH = args.multispec if args.multispec else None

# Initialize Metashape document
doc = Metashape.Document()
if args.proj_path:
    proj_file = args.proj_path
    doc.open(proj_file, read_only=False)  # Open the document in editable mode
else:
    proj_file = doc.path

if not proj_file:
    site = args.site
    date = args.date
    proj_file = os.path.join(BASE_DIR, site, date, f"{site}_{date}_metashape.psx")
    if not os.path.exists(proj_file):
        doc.save(proj_file)
    doc.open(proj_file, read_only=False)  # Open the document in editable mode
    doc.save(proj_file)

# Validate paths
if not MRK_PATH:
    MRK_PATH = Path(proj_file).parents[1] / "rgb/level0_raw"
    if not MRK_PATH.is_dir():
        sys.exit(f"{MRK_PATH} directory does not exist. Check and input paths using -rgb")
    MRK_PATH = str(MRK_PATH)

if not MICASENSE_PATH:
    MICASENSE_PATH = Path(proj_file).parents[1] / "multispec/level0_raw"
    if not MICASENSE_PATH.is_dir():
        sys.exit(f"{MICASENSE_PATH} directory does not exist. Check and input paths using -multispec")
    MICASENSE_PATH = str(MICASENSE_PATH)

if args.drtk and not Path(args.drtk).is_file():
    sys.exit(f"{args.drtk} file does not exist. Check and input correct path using -drtk option")

if args.smooth not in DICT_SMOOTH_STRENGTH:
    sys.exit("Value for -smooth must be one of low, medium or high.")

# Set quality values for the downscale value in RGB and Multispec for testing
quality1 = 4 if args.test else 1
quality2 = 8 if args.test else 4
quality3 = 4 if args.test else 1

# Export blockshifted P1 positions. Not used in script. Useful for debug or to restart parts of script following any issues.
P1_CAM_CSV = Path(proj_file).parent / "dbg_shifted_p1_pos.csv"
# By default save the CSV with updated MicaSense positions in the MicaSense folder. CSV used within script.
MICASENSE_CAM_CSV = Path(proj_file).parent / "interpolated_micasense_pos.csv"

# Add Images to Project
Add RGB and multispectral images to the Metashape project, create chunks, and set up the project structure.

In [9]:
# Add RGB and multispectral images to the Metashape project, create chunks, and set up the project structure.

# Function to add images to the project
def add_images_to_project():
    global doc, MRK_PATH, MICASENSE_PATH

    # Add RGB images
    p1_images = find_files(MRK_PATH, (".jpg", ".jpeg", ".tif", ".tiff"))
    chunk_rgb = doc.addChunk()
    chunk_rgb.label = CHUNK_RGB
    chunk_rgb.addPhotos(p1_images)

    # Check that chunk is not empty and images are in default WGS84 CRS
    if len(chunk_rgb.cameras) == 0:
        sys.exit("Chunk rgb empty")
    if "EPSG::4326" not in str(chunk_rgb.crs):
        sys.exit("Chunk rgb: script expects images loaded to be in CRS WGS84 EPSG::4326")

    # Add multispectral images
    micasense_images = find_files(MICASENSE_PATH, (".jpg", ".jpeg", ".tif", ".tiff"))
    chunk_multispec = doc.addChunk()
    chunk_multispec.label = CHUNK_MULTISPEC
    chunk_multispec.addPhotos(micasense_images)
    doc.save()

# Execute the function to add images to the project
add_images_to_project()

OSError: Document.save(): editing is disabled in read-only mode

In [16]:
# Check if required chunks are present in the project
check_chunk_list = [CHUNK_RGB, CHUNK_MULTISPEC]
dict_chunks = {}
for get_chunk in doc.chunks:
    dict_chunks.update({get_chunk.label: get_chunk.key})

chunk = doc.findChunk(dict_chunks[CHUNK_RGB])
if not chunk:
    sys.exit("Chunk rgb not found in the project")

chunk = doc.findChunk(dict_chunks[CHUNK_MULTISPEC])
if not chunk:
    sys.exit("Chunk multispec not found in the project")

# Resume Processing
Define the `resume_proc` function to resume processing after user input on calibration images and call the main processing functions.

In [ ]:
def resume_proc():
    """
    Resume processing after user input on calibration images.
    """
    # Process RGB chunk if multionly is not set
    if not args.multionly:
        proc_rgb()
    # Process multispec chunk
    proc_multispec()
    print("End of script")

# Execute the resume_proc function to start processing
resume_proc()